In [7]:
# Import required libraries
import pandas as pd
import requests
import time

In [8]:
# Assign the seasons accordingly
season_mapping = {
    '2023-2024': 20720,
    '2022-2023' : 17664,
    '2021-2022' : 16390,
    '2020-2021' : 15382,
    '2019-2020' : 14022
}

In [9]:
# Function takes the season, performs data cleaning and returns ratings
def ratings(season):

    # Scrape data using API

    headers = {
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'Referer': 'https://www.fotmob.com/leagues/47/stats/season/20720/players/rating/team/9825',
    'x-fm-req': 'eyJib2R5Ijp7ImNvZGUiOjE3MDYxMzExOTE5Mjl9LCJzaWduYXR1cmUiOiJCNjg0Mjg0ODFEQTY5NjQ3MEVCNTFFQjkyM0RFNjcyQSJ9',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'sec-ch-ua-platform': '"Windows"',
    }

    params = {
        'id': '47',
        'season': season_mapping[season],
        'type': 'players',
        'stat': 'rating',
        'teamId': '9825',
    }

    response = requests.get('https://www.fotmob.com/api/leagueseasondeepstats', params=params, headers=headers)

    # Check the response status code 
    if response.status_code == 200:
        playerRatings = response.json()

        statsData = playerRatings['statsData']
        names = [player['nameAndSubstatValue']['name'] for player in statsData] 

        
        # Convert the dictionary into a dataframe
        df_ratings = pd.DataFrame.from_dict(statsData)
        df_ratings['nameAndSubstatValue'] = names
        df_ratings = df_ratings.loc[:,['nameAndSubstatValue','statValue','rank']]
        df_ratings['season_name'] = season

        return df_ratings
    
    else:
        f'Wrong response: {response.status_code}'
    

### Historical Data

In [10]:
# Save old season table

# Run for each year   
df_ratings_19 = ratings('2019-2020')
df_ratings_20 = ratings('2020-2021')
df_ratings_21 = ratings('2021-2022')
df_ratings_22 = ratings('2022-2023')

# Join all the old data together into a table,
previous_season = [df_ratings_19,df_ratings_20,df_ratings_21,df_ratings_22]
df_ratings_1922_table = pd.concat(previous_season, ignore_index=True)

# Export table to .csv
# Export values
df_ratings_1922_table.to_csv("/Users/Razak/Desktop/Arsenal Database Project/scraped csv files/Raw csv/Old Season/temp_ratings19-22_raw.csv", index=False, encoding = 'utf-8-sig')

### Current Season Data
Run the code below to add new data for the current season to the existing data above, then export the updated data.

In [11]:
# save new season table (Always updating)
df_new = ratings('2023-2024')

# read old season table (using pandas read excel)
previous_table = pd.read_csv("/Users/Razak/Desktop/Arsenal Database Project/scraped csv files/Raw csv/Old Season/temp_ratings19-22_raw.csv")

# join new and onld season (concat)
joined_season = pd.concat([previous_table, df_new], ignore_index=True)

# export join (export and save as old+new updated table)
joined_season.to_csv("/Users/Razak/Desktop/Arsenal Database Project/scraped csv files/Raw csv/Updated tables/temp_updatedratings_raw.csv", index=False, encoding = 'utf-8-sig')